In [1]:
!pip install --upgrade pip mlflow transformers datasets boto3 torch tensorflow huggingface_hub evaluate tqdm tf-keras peft 


In [2]:
! pip install -U accelerate>=0.26.0

In [4]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    TextStreamer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch, gc, os
import numpy as np
# from unsloth.chat_templates import get_chat_template
# from unsloth import FastLanguageModel, is_bfloat16_supported
from tqdm import tqdm

print(torch.cuda.is_available())
torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

2025-04-09 02:26:04.747528: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744165564.764707    3759 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744165564.770016    3759 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744165564.782417    3759 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744165564.782429    3759 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744165564.782431    3759 computation_placer.cc:177] computation placer alr

In [5]:
tqdm.pandas()

In [90]:
from huggingface_hub import notebook_login
notebook_login()
# !huggingface-cli login --token 

In [7]:
dataset = load_dataset("hpe-ai/medical-cases-classification-tutorial")
dataset

Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['description', 'transcription', 'sample_name', 'medical_specialty', 'keywords'],
        num_rows: 1724
    })
    validation: Dataset({
        features: ['description', 'transcription', 'sample_name', 'medical_specialty', 'keywords'],
        num_rows: 370
    })
    test: Dataset({
        features: ['description', 'transcription', 'sample_name', 'medical_specialty', 'keywords'],
        num_rows: 370
    })
})

In [8]:
# Columns to keep
columns_to_keep = ["transcription", "medical_specialty"]

# Remove all columns that are not in `columns_to_keep`
dataset = dataset.remove_columns(
    [col for col in dataset["train"].column_names if col not in columns_to_keep]
)

dataset["validation"] = dataset["validation"].remove_columns(
    [col for col in dataset["validation"].column_names if col not in columns_to_keep]
)

dataset["test"] = dataset["test"].remove_columns(
    [col for col in dataset["test"].column_names if col not in columns_to_keep]
)


In [10]:
# dataset = dataset.remove_columns([col for col in dataset.column_names if col not in ["transcription", "medical_specialty"]])
medical_test = dataset['test']
medical_train = dataset['train']
medical_val = dataset['validation']


In [11]:
train_df = medical_test.to_pandas()
train_df.head()

,transcription,medical_specialty
0,"PREOPERATIVE DIAGNOSIS:, Dural tear, postoper...",Orthopedic
1,"REASON FOR CONSULTATION: , Loculated left effu...",Cardiovascular / Pulmonary
2,"REASON FOR CONSULTATION: ,Abnormal echocardio...",Cardiovascular / Pulmonary
3,"PREOPERATIVE DIAGNOSES: , Cervical spondylosis...",Orthopedic
4,"CC:, Weakness.,HX:, This 30 y/o RHM was in goo...",Neurology


In [12]:
refined_train = train_df[columns_to_keep]

In [13]:
refined_train.medical_specialty.value_counts()

medical_specialty
Cardiovascular / Pulmonary    109
Orthopedic                     57
Neurology                      47
Gastroenterology               41
Obstetrics / Gynecology        32
Neurosurgery                   20
Hematology - Oncology          15
ENT - Otolaryngology           14
Ophthalmology                  11
Nephrology                     10
Psychiatry / Psychology         9
Pediatrics - Neonatal           3
Radiology                       2
Name: count, dtype: int64

## Loading the Model

In [14]:
# Use a pipeline to test the model
from transformers import pipeline
pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-1B")

Device set to use cuda:0


In [75]:
pipe("how are you?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'how are you? today we are going to see the new samsung galaxy tab 3 7.0, the tab'}]

In [76]:
model_checkpoint = 'meta-llama/Llama-3.2-1B'
# Label Maps
label2id = {
    "Cardiovascular / Pulmonary": 0,
    "Orthopedic": 1,
    "Neurology": 2,
    "Gastroenterology": 3,
    "Obstetrics / Gynecology": 4,
    "Neurosurgery": 5,
    "Hematology - Oncology": 6,
    "ENT - Otolaryngology": 7,
    "Ophthalmology": 8,
    "Nephrology": 9,
    "Psychiatry / Psychology": 10,
    "Pediatrics - Neonatal": 11,
    "Radiology": 12
}


id2label = {v: k for k, v in label2id.items()}

print("label2id:", label2id)
print("id2label:", id2label)


# classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, 
    num_labels=13, 
    id2label=id2label, 
    label2id=label2id,
    device_map="auto",
    torch_dtype=torch.float32)


label2id: {'Cardiovascular / Pulmonary': 0, 'Orthopedic': 1, 'Neurology': 2, 'Gastroenterology': 3, 'Obstetrics / Gynecology': 4, 'Neurosurgery': 5, 'Hematology - Oncology': 6, 'ENT - Otolaryngology': 7, 'Ophthalmology': 8, 'Nephrology': 9, 'Psychiatry / Psychology': 10, 'Pediatrics - Neonatal': 11, 'Radiology': 12}
id2label: {0: 'Cardiovascular / Pulmonary', 1: 'Orthopedic', 2: 'Neurology', 3: 'Gastroenterology', 4: 'Obstetrics / Gynecology', 5: 'Neurosurgery', 6: 'Hematology - Oncology', 7: 'ENT - Otolaryngology', 8: 'Ophthalmology', 9: 'Nephrology', 10: 'Psychiatry / Psychology', 11: 'Pediatrics - Neonatal', 12: 'Radiology'}


Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [57]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': "<pad>"})
    model.resize_token_embeddings(len(tokenizer))

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id


In [58]:
tokenizer.pad_token

'<|end_of_text|>'

In [59]:

# create tokenize function
def tokenize_function(data):
    # extract text
    text = data["transcription"]
    label = [label2id[val] for val in data["medical_specialty"]]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        padding="max_length",
        max_length=512
    )
    tokenized_inputs["labels"] = label

    return tokenized_inputs

In [60]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset


DatasetDict({
    train: Dataset({
        features: ['transcription', 'medical_specialty', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1724
    })
    validation: Dataset({
        features: ['transcription', 'medical_specialty', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 370
    })
    test: Dataset({
        features: ['transcription', 'medical_specialty', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 370
    })
})

In [61]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Check for Accuracy for current dataset

In [62]:
accuracy_metrics = evaluate.load("accuracy")

In [63]:
def calculate_metrics(preds):
    predictions, labels  = preds
    return f"Accuracy is {accuracy_metrics.compute(predictions=predictions, references= labels)}"

def get_metrics(preds):
    logits, labels = preds
    predictions = torch.argmax(torch.tensor(logits), dim=1)

    return accuracy_metrics.compute(
        predictions=predictions.tolist(),
        references=labels.tolist()
    )


### Testing Base Model

In [77]:
test_df = medical_test.to_pandas()[columns_to_keep]
test_df.head()

,transcription,medical_specialty
0,"PREOPERATIVE DIAGNOSIS:, Dural tear, postoper...",Orthopedic
1,"REASON FOR CONSULTATION: , Loculated left effu...",Cardiovascular / Pulmonary
2,"REASON FOR CONSULTATION: ,Abnormal echocardio...",Cardiovascular / Pulmonary
3,"PREOPERATIVE DIAGNOSES: , Cervical spondylosis...",Orthopedic
4,"CC:, Weakness.,HX:, This 30 y/o RHM was in goo...",Neurology


In [82]:
def call_model(rows, model=model):
  text = rows['transcription']
  inputs = tokenizer.encode(text, return_tensors="pt")
  inputs = inputs.to(model.device)

  #Compute Logits
  logits = model(inputs).logits

  #Convert logits to label
  predictions = torch.argmax(logits, dim=1)

  return id2label[predictions.item()]

# test_df['Base_Model_Output'] = test_df.progress_apply(call_model, axis=1)

In [80]:
test_df['Base_Model_Output_Id'] = test_df['Base_Model_Output'].apply(lambda x: label2id[x])
test_df['medical_specialty_id'] = test_df['medical_specialty'].apply(lambda x: label2id[x])
# import pandas as pd
# test_df = pd.read_csv('base_model_output.csv')
print(calculate_metrics((test_df['Base_Model_Output_Id'].tolist(), test_df['medical_specialty_id'].tolist())))

test_df.to_csv('base_model_output.csv')


Accuracy is {'accuracy': 0.002702702702702703}


Since we are not getting good accuracies on the base model, let's try to fine tune the model, but use Parameter Efficient Fine Tuning (PEfT) using Low Rank Apadtation Matrices (LoRA), so as to avoid bigger model size, or catastrophic forgetting, we'll have lower parameters to train the model on, with it's original weights intact.

## Model Fine Tuning

In [64]:
peft_config = LoraConfig(task_type="SEQ_CLS", # Using for Sequence Classification
                        r=4, # keeping the rank small because of faster training
                        lora_alpha=32, # Keeping it 32 for efficient Training
                        lora_dropout=0.05, # Making sure it doesn't overfit
                        use_rslora=True, # For efficient Training, Helps with convergence
                        target_modules = ['q_lin', 'v_proj'])  # Good performance/cost trade-off

In [65]:
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 190,464 || all params: 1,236,033,536 || trainable%: 0.0154


In [33]:
import subprocess

subprocess.run(["accelerate", "config", "default"], input=b"\n".join([
    b"0",       # This machine
    b"1",       # Multi-GPU
    b"8",       # Number of processes (GPUs)
    b"no",      # Use DeepSpeed: no
]), check=True)


Configuration already exists at /root/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.


CompletedProcess(args=['accelerate', 'config', 'default'], returncode=0)

In [66]:
learning_rate = 1e-3
batch_size = 4
num_epochs = 10

training_args = TrainingArguments(
    output_dir="./checkpoint",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    fp16=False,
    bf16=False,
    gradient_accumulation_steps=4,
    save_steps=500,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    eval_accumulation_steps = 1,
    dataloader_pin_memory=True,
    dataloader_num_workers=2,

)



In [67]:
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=get_metrics,
)


/tmp/ipykernel_3759/3879297496.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [31]:
torch.cuda.empty_cache()

In [68]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,2.129500,1.379465,0.727027
2,1.094700,0.867492,0.775676
3,0.776600,0.726977,0.789189
4,0.641700,0.636631,0.797297
5,0.525600,0.704872,0.808108
6,0.462100,0.862990,0.802703
7,0.393300,0.791373,0.816216
8,0.336800,0.785252,0.791892
9,0.240200,0.949894,0.783784


/venv/main/lib/python3.10/site-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/venv/main/lib/python3.10/site-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/venv/main/lib/python3.10/site-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/venv/main/lib/python3.10/site-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/venv/main/lib/python3.10/site-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetunin

TrainOutput(global_step=1070, training_loss=0.6567519562266697, metrics={'train_runtime': 482.188, 'train_samples_per_second': 35.754, 'train_steps_per_second': 2.219, 'total_flos': 5.10840941838336e+16, 'train_loss': 0.6567519562266697, 'epoch': 9.909512761020881})

##### As we can see, the validation loss started increasing even after applying L2 regularization and Dropout layer. This means the model started to overfit after epoch 5. We further need to apply early stopping and decrease the dropour layer. 

In [70]:
from transformers import EarlyStoppingCallback

# EarlyStoppingCallback to stop training if validation loss does not improve
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=2,  # Wait for 2 epochs of no improvement
    early_stopping_threshold=0.01  # The minimum improvement required is 0.01
)


In [71]:
peft_config_v2 = LoraConfig(task_type="SEQ_CLS", # Using for Sequence Classification
                        r=4, # keeping the rank small because of faster training
                        lora_alpha=32, # Keeping it 32 for efficient Training
                        lora_dropout=0.01, # Making sure it doesn't overfit
                        use_rslora=True, # For efficient Training, Helps with convergence
                        target_modules = ['q_lin', 'v_proj'])  # Good performance/cost trade-off
peft_model_v2 = get_peft_model(model, peft_config_v2)
peft_model_v2.print_trainable_parameters()

trainable params: 190,464 || all params: 1,236,033,536 || trainable%: 0.0154


/venv/main/lib/python3.10/site-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/venv/main/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [73]:
learning_rate = 1e-3
batch_size = 4
num_epochs = 10

training_args = TrainingArguments(
    output_dir="./checkpoint",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    fp16=False,
    bf16=False,
    gradient_accumulation_steps=4,
    save_steps=500,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",  # Metric to monitor for early stopping
    eval_accumulation_steps = 1,
    dataloader_pin_memory=True,
    dataloader_num_workers=2,

)


trainer = Trainer(
    model=peft_model_v2,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=get_metrics,
    callbacks=[early_stopping]
)


/tmp/ipykernel_3759/1309950691.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [74]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.221300,1.465908,0.659459
2,0.868900,0.798287,0.770270
3,0.636200,0.851329,0.737838
4,0.506600,0.844056,0.767568


/venv/main/lib/python3.10/site-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/venv/main/lib/python3.10/site-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/venv/main/lib/python3.10/site-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/venv/main/lib/python3.10/site-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=432, training_loss=0.7900345325469971, metrics={'train_runtime': 193.9036, 'train_samples_per_second': 88.91, 'train_steps_per_second': 5.518, 'total_flos': 2.06202243907584e+16, 'train_loss': 0.7900345325469971, 'epoch': 4.0})

#### Testing PEFT Model

In [84]:
# peft_model_v2
test_df['Fine_Tuned_Model_Output'] = test_df.progress_apply((lambda row: call_model(row, peft_model_v2)), axis=1)
test_df['Fine_Tuned_Model_Output_Id'] = test_df['Fine_Tuned_Model_Output'].apply(lambda x: label2id[x])
# test_df['medical_specialty_id'] = test_df['medical_specialty'].apply(lambda x: label2id[x])
# import pandas as pd
# test_df = pd.read_csv('base_model_output.csv')
print(calculate_metrics((test_df['Fine_Tuned_Model_Output_Id'].tolist(), test_df['medical_specialty_id'].tolist())))

test_df.to_csv('Fine_Tuned_model_output.csv')


100%|██████████| 370/370 [00:10<00:00, 34.25it/s]

Accuracy is {'accuracy': 0.7648648648648648}


Base Model's Accuracy was {'accuracy': 0.27%} 

Fine Tuned Model's Accuracy is {'accuracy': 76.4%}

After Fine Tuning the model, we have increased the accuracies by ~76.2%


#### Pushing the model to HuggingFace

In [88]:
model_checkpoint.split("/")[1]

'Llama-3.2-1B'

In [91]:
hf_name = 'adityatandon94' 
model_id = hf_name + "/" + model_checkpoint.split("/")[1] + "-peft-medical-specialty-classification" 
peft_model_v2.push_to_hub(model_id) # save model

/venv/main/lib/python3.10/site-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/adityatandon94/Llama-3.2-1B-peft-medical-specialty-classification/commit/8d30d7df88293c1c9e18cca2c6c0b6ba31e6947c', commit_message='Upload model', commit_description='', oid='8d30d7df88293c1c9e18cca2c6c0b6ba31e6947c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/adityatandon94/Llama-3.2-1B-peft-medical-specialty-classification', endpoint='https://huggingface.co', repo_type='model', repo_id='adityatandon94/Llama-3.2-1B-peft-medical-specialty-classification'), pr_revision=None, pr_num=None)

In [92]:
trainer.push_to_hub(model_id)

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/adityatandon94/checkpoint/commit/a7baef521e6bcaa7ee8af65aa6d6c9202cc6d8d0', commit_message='adityatandon94/Llama-3.2-1B-peft-medical-specialty-classification', commit_description='', oid='a7baef521e6bcaa7ee8af65aa6d6c9202cc6d8d0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/adityatandon94/checkpoint', endpoint='https://huggingface.co', repo_type='model', repo_id='adityatandon94/checkpoint'), pr_revision=None, pr_num=None)

### To Kill Processes

In [46]:
!nvidia-smi
# !kill -9 1659
# # !ps aux | grep python
# # !pkill -f python
# !kill -9 $(nvidia-smi | grep python | awk '{print $5}')


Wed Apr  9 02:09:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.58.02              Driver Version: 555.58.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        On  |   00000000:01:00.0 Off |                  Off |
|  0%   34C    P8             13W /  385W |   12706MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [56]:
# trainer.train(resume_from_checkpoint=True)